In [ ]:
#pip install fastapi

In [ ]:
#pip install uvicorn

In [ ]:
#pip install snowflake-connector-python

In [ ]:
#pip install passlib

In [ ]:
#pip install "pydantic[email]"

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, EmailStr
from snowflake.connector import connect
from passlib.context import CryptContext
import uvicorn

app = FastAPI()

# Snowflake Database Configuration
SNOWFLAKE_USER = "VEDANT1511"
SNOWFLAKE_PASSWORD = "Ronaldo7vedant"
SNOWFLAKE_ACCOUNT = "GMBDPCM-ZEB88340"
SNOWFLAKE_DATABASE = "STACKAI"
SNOWFLAKE_SCHEMA = "PUBLIC"

# Password Hashing Configuration
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")


# Function to create a Snowflake connection
def get_snowflake_connection():
    conn = connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA
    )
    return conn


# Pydantic model for user signup
class UserSignup(BaseModel):
    email: EmailStr
    username: str
    password: str


# New endpoint for manual user signup
@app.post("/manual-register/")
async def manual_register_user(user_signup: UserSignup):
    # Hash the password before storing it
    hashed_password = pwd_context.hash(user_signup.password)

    # Store the user information in the database
    try:
        conn = get_snowflake_connection()
        cursor = conn.cursor()

        cursor.execute(
            f"INSERT INTO users (email, username, password) VALUES ('{user_signup.email}', '{user_signup.username}', '{hashed_password}')"
        )

        cursor.close()
        conn.commit()
        conn.close()

        # Return the user data in the response
        return {"message": "User registered successfully!", "email": user_signup.email, "username": user_signup.username}
    except Exception as e:
        # Handle database errors and other exceptions
        raise HTTPException(status_code=500, detail="Internal Server Error")


if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
Do this in Postman: Create a post request as:
http://localhost:8000/register/

{
  "email": "user@example.com",
  "username": "new_user",
  "password": "secretpassword"
}